In [261]:
from itertools import combinations
from z3 import *
from utils import *
import math

### naive pairwise


In [262]:
def at_least_one_np(bool_vars):
    return Or(bool_vars)

def at_most_one_np(bool_vars):
    return [Not(And(pair[0], pair[1])) for pair in combinations(bool_vars, 2)]

def exactly_one_np(bool_vars):
    return And(at_least_one_np(bool_vars), And(at_most_one_np(bool_vars)))



def at_least_k_np(bool_vars, k):
    return at_most_k_np([Not(var) for var in bool_vars], len(bool_vars)-k)

def at_most_k_np(bool_vars, k):
    return And([Or([Not(x) for x in X]) for X in combinations(bool_vars, k + 1)])

def exactly_k_np(bool_vars, k):
    return And(at_most_k_np(bool_vars, k), at_least_k_np(bool_vars, k))

### sequential

In [263]:
def at_least_one_seq(bool_vars):
    return at_least_one_np(bool_vars)

def at_most_one_seq(bool_vars):
    constraints = []
    n = len(bool_vars)
    s = [Bool(f"s_{i}") for i in range(n - 1)]
    constraints.append(Or(Not(bool_vars[0]), s[0]))
    constraints.append(Or(Not(bool_vars[n-1]), Not(s[n-2])))
    for i in range(1, n - 1):
        constraints.append(Or(Not(bool_vars[i]), s[i]))
        constraints.append(Or(Not(bool_vars[i]), Not(s[i-1])))
        constraints.append(Or(Not(s[i-1]), s[i]))
    return And(constraints)

def exactly_one_seq(bool_vars):
    return And(at_least_one_seq(bool_vars), at_most_one_seq(bool_vars))



def at_least_k_seq(bool_vars, k):
    return at_most_k_seq([Not(var) for var in bool_vars], len(bool_vars)-k)

def at_most_k_seq(bool_vars, k):
    constraints = []
    n = len(bool_vars)
    s = [[Bool(f"s_{i}_{j}") for j in range(k)] for i in range(n - 1)]
    constraints.append(Or(Not(bool_vars[0]), s[0][0]))
    constraints += [Not(s[0][j]) for j in range(1, k)]
    for i in range(1, n-1):
        constraints.append(Or(Not(bool_vars[i]), s[i][0]))
        constraints.append(Or(Not(s[i-1][0]), s[i][0]))
        constraints.append(Or(Not(bool_vars[i]), Not(s[i-1][k-1])))
        for j in range(1, k):
            constraints.append(Or(Not(bool_vars[i]), Not(s[i-1][j-1]), s[i][j]))
            constraints.append(Or(Not(s[i-1][j]), s[i][j]))
    constraints.append(Or(Not(bool_vars[n-1]), Not(s[n-2][k-1])))   
    return And(constraints)

def exactly_k_seq(bool_vars, k):
    return And(at_most_k_seq(bool_vars, k), at_least_k_seq(bool_vars, k))

### bitwise

In [264]:
def toBinary(num, length = None):
    num_bin = bin(num).split("b")[-1]
    if length:
        return "0"*(length - len(num_bin)) + num_bin
    return num_bin
    
def at_least_one_bw(bool_vars):
    return at_least_one_np(bool_vars)

def at_most_one_bw(bool_vars):
    constraints = []
    n = len(bool_vars)
    m = math.ceil(math.log2(n))
    r = [Bool(f"r_{i}") for i in range(m)]
    binaries = [toBinary(i, m) for i in range(n)]
    for i in range(n):
        for j in range(m):
            phi = Not(r[j])
            if binaries[i][j] == "1":
                phi = r[j]
            constraints.append(Or(Not(bool_vars[i]), phi))        
    return And(constraints)

def exactly_one_bw(bool_vars):
    return And(at_least_one_bw(bool_vars), at_most_one_bw(bool_vars)) 

### heule

In [265]:
def at_least_one_he(bool_vars):
    return at_least_one_np(bool_vars)

def at_most_one_he(bool_vars):
    if len(bool_vars) <= 4:
        return And(at_most_one_np(bool_vars))
    y = Bool(f"y")
    return And(And(at_most_one_np(bool_vars[:3] + [y])), And(at_most_one_he(bool_vars[3:] + [Not(y)])))

def exactly_one_he(bool_vars):
    return And(at_most_one_he(bool_vars), at_least_one_he(bool_vars))

# Model

#### get the data

In [329]:
def read_file(in_file):
    f = open(in_file, "r")
    lines = f.read().splitlines()
    
    w = int(lines[0])
    n = int(lines[1])
    
    x = []
    y = []
    for i in range(int(n)):
            split = lines[i + 2].split(' ')
            x.append(int(split[0]))
            y.append(int(split[1]))
            
    maxlen = sum(y)
    return w, n, x, y, maxlen

w, n, x, y, maxlen = read_file(".\instances\ins-1.txt")

print(w,n,x,y,maxlen)

8 4 [3, 3, 5, 5] [3, 5, 3, 5] 16


#### define the variables

In [330]:
solution = [[[Bool(f"solution_{i}_{j}_{k}") for k in range(n)] for j in range(maxlen)] for i in range(w)]   
#k is the number of the silicon
#i is the possible positions in w
#j is the possible positions in l

# length of the plate to minimize (one-hot representation)
l = [Bool(f"l_{i}") for i in range(maxlen)]

s = Solver()

#### choice of the method

In [331]:
def at_least_one(bool_vars):
    return Or(bool_vars)

def at_most_one(bool_vars):
    return [Not(And(pair[0], pair[1])) for pair in combinations(bool_vars, 2)]

def exactly_one(bool_vars):
    return at_most_one(bool_vars) + [at_least_one(bool_vars)]

#### constraints

In [332]:
#no overlapping
for j in range(maxlen):
    for i in range(w):
        s.add(at_most_one([solution[i][j][k] for k in range(n)]))
            
#makes sure silicons fit            
for k in range(n):
    possible_sols = []
    for i in range(w-x[k]+1):
        for j in range(maxlen-y[k]+1):
            circuit = []
            for ox in range(w):
                for oy in range(maxlen):
                    if i<=ox<i+x[k] and j<=oy<j+y[k]:
                        circuit.append(solution[ox][oy][k])
                    else:
                        circuit.append(Not(solution[ox][oy][k]))
            possible_sols.append(And(circuit))
    s.add(exactly_one(possible_sols))
            
#for every silicon, it has exactly one position
#for k in range(n):
#    for j in range(maxlen):
#        s.add(exactly_one([solution[i][j][k] for i in range(w)]))
        
#for k in range(n):
#    for i in range(w):
#        s.add(at_most_one([solution[i][j][k] for j in range(maxlen)]))

In [333]:
if s.check() == sat:
    print(s.model())
else:
    print("Failed to solve")

[solution_2_6_0 = False,
 solution_1_12_1 = False,
 solution_3_2_0 = False,
 solution_7_14_0 = False,
 solution_1_0_2 = False,
 solution_3_12_1 = False,
 solution_0_15_0 = False,
 solution_4_6_1 = False,
 solution_6_13_2 = False,
 solution_0_11_1 = False,
 solution_3_2_3 = False,
 solution_1_2_1 = False,
 solution_3_11_2 = False,
 solution_0_0_2 = False,
 solution_1_6_2 = False,
 solution_7_1_2 = False,
 solution_0_12_3 = False,
 solution_5_9_1 = False,
 solution_3_15_0 = False,
 solution_6_9_2 = False,
 solution_6_8_1 = False,
 solution_5_13_1 = False,
 solution_5_10_2 = False,
 solution_6_3_2 = False,
 solution_7_5_0 = False,
 solution_6_11_1 = False,
 solution_2_9_3 = False,
 solution_3_1_1 = False,
 solution_4_8_1 = False,
 solution_2_12_1 = False,
 solution_2_3_1 = True,
 solution_1_7_3 = True,
 solution_6_5_2 = False,
 solution_6_15_3 = False,
 solution_6_2_3 = False,
 solution_5_4_1 = False,
 solution_0_1_1 = False,
 solution_0_13_3 = False,
 solution_1_3_3 = True,
 solution_4_1

In [338]:
def get_solution(model, solution, w, l, n, r=None):
    # Create solution array
    solution = np.array([[[is_true(model[solution[i][j][k]]) for k in range(n)] for j in range(maxlen)] for i in range(w)])
    p_x_sol = []
    p_y_sol = []
    rot_sol = [False for i in range(n)]

    for k in range(n):
        x_ids, y_ids = solution[:, :, k].nonzero()
        x = np.min(x_ids)
        y = np.min(y_ids)
        p_x_sol.append(x)
        p_y_sol.append(y)
        if r is not None:
            rot_sol[k] = is_true(model[r[k]])
    return p_x_sol, p_y_sol, rot_sol

In [339]:
get_solution(s.model(), solution, w, maxlen, n, r=None)

[[[False False False False]
  [False False  True False]
  [False False  True False]
  [False  True  True False]
  [False  True False False]
  [False  True False False]
  [False  True False False]
  [False  True False False]
  [False False False False]
  [False False False False]
  [False False False False]
  [False False False False]
  [False False False False]
  [False False False False]
  [False False False False]
  [False False False False]]

 [[False False False False]
  [False False  True False]
  [False False  True False]
  [False  True  True  True]
  [False  True False  True]
  [False  True False  True]
  [False  True False  True]
  [False  True False  True]
  [False False False False]
  [False False False False]
  [False False False False]
  [False False False False]
  [False False False False]
  [False False False False]
  [False False False False]
  [False False False False]]

 [[False False False False]
  [False False  True False]
  [False False  True False]
  [False  True  

([2, 0, 0, 1], [11, 3, 1, 3], [False, False, False, False])